In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [78]:
df = pd.read_csv("shuffled_data.csv")
X = df[df.columns[-1]]
y = df[df.columns[1:-1]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False)

In [4]:
from transformers import AutoTokenizer, BertTokenizer, AutoModel


# initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('./models/GilBERTo-uncased-from-camembert')

In [94]:
dataset = []
for idx, row in df.iterrows():
    dict_row = tokenizer(row['sentence'].lower())
    dict_row['sentence_id'] = row['sentence_id']
    dataset.append(dict_row)

In [95]:
dataset_df = pd.DataFrame(dataset)
dataset_df = dataset_df.merge(df, on='sentence_id')
dataset_df.to_pickle('./dataset_dataframe_test.pkl')

In [7]:
dataset_df = pd.read_pickle('./dataset_dataframe.pkl')

In [2]:
from transformers import TFAutoModel

pre_trained = TFAutoModel.from_pretrained('./models/GilBERTo-uncased-from-camembert', from_pt=True)
pre_trained.trainable = False

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFCamembertModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFCamembertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFCamembertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFCamembertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.


In [75]:
import keras

class Smart_Batch_Padding(keras.layers.Layer):
    def call(self, inputs):
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        seq_lengths = tf.math.reduce_sum(attention_mask, axis=1)
        max_seq_len = tf.math.reduce_max(seq_lengths)

        return {'input_ids': input_ids[:,:max_seq_len],
                'attention_mask': attention_mask[:,:max_seq_len]}

In [84]:
import tensorflow as tf
import numpy as np
X = {}

input_ids = np.array(dataset_df['input_ids'].to_list())
attention_mask = np.array(dataset_df['attention_mask'].to_list())

X['input_ids'] = tf.constant(input_ids)
X['attention_mask'] = tf.constant(attention_mask)

In [87]:
target_cols = dataset_df.columns[3:-1]
y = tf.constant(dataset_df[target_cols])

In [ ]:
dict1 = {'input_ids': tf.constant([1,2,3]), 'input_ids': tf.constant([1,2,3])}

In [100]:
class NewModel(tf.keras.Model):
    def __init__(self, pre_trained, n_outputs):
        super(NewModel, self).__init__()
        self.smart_batch_padding = Smart_Batch_Padding()
        self.pre_trained = pre_trained
        self.dense = tf.keras.layers.Dense(n_outputs)
        self.sigmoid = 


    def call(self, inputs):
        x = self.smart_batch_padding(inputs)
        x = self.pre_trained(x)[1]
        x = self.dense(x)
        return x

new_model = NewModel(pre_trained, 24)
# new_model(input1)

In [106]:
optimizer = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metric = tf.keras.metrics.AUC(from_logits=True)

new_model.compile(optimizer=optimizer, loss=loss, metrics=metric)

In [107]:
history = new_model.fit(X, y, batch_size=16, validation_split=0.2, epochs=30)

Epoch 1/30
100/317 [========>.....................] - ETA: 5:32 - loss: 0.6936 - auc_1: 0.5500

KeyboardInterrupt: 